In [1]:
import pandas as pd
import numpy as np

In [2]:
# 유저테이블 불러오기
click = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/유저테이블.csv")

# media portfolio 불러오기
media_portfolio = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv")

# pool 불러오기
pool = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/ads_pool.csv")

In [29]:
new = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/신규가상광고.csv")

In [38]:
pool.head()

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
0,0,0,6508,1,1,2,0,10,금융,7,2381500,NaN,NaN,NONE,1400,1100,2025-07-29 08:54:30,2025-07-29 08:55:55,스마트피싱보호,2020-04-23 17:00:00,0,1,0.0,0.0,300,0.3,0,MEDIUM,2.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
1,1,1,6985,2,30,46,0,7,금융,7,1201938,NaN,NaN,NONE,2700,2300,2025-07-26 11:02:18,2025-08-24 05:39:13,DB손해보험 다이렉트,2020-06-30 15:00:00,0,29,0.0,0.0,400,0.2,0,MEDIUM,1.586207,1.034483,MEDIUM,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0,NaN,0.0,0.0
2,2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,NaN,NaN,NONE,3200,2420,2025-08-02 08:30:52,2025-08-18 22:32:25,파일썬,2020-11-10 11:21:00,0,17,0.0,0.0,780,0.3,0,MEDIUM,0.647059,0.352941,SMALL,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
3,3,3,9264,1,1,1,0,10,금융,7,2372800,NaN,NaN,NONE,11000,8250,2025-08-14 13:54:12,2025-08-14 13:54:12,하나 가득담은 운전자보험가입,2020-11-30 00:00:00,0,1,0.0,0.0,2750,0.3,0,LARGE,1.000000,1.000000,SMALL,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
4,4,4,9716,2,4,6,1,8,생활,2,2699900,74.0,74.0,NONE,600,336,2025-07-28 01:22:36,2025-07-29 07:14:36,베이비러브 참여 신청,2021-01-11 16:00:00,0,2,0.5,0.2,264,0.8,264,MEDIUM,3.000000,2.000000,MEDIUM,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0,NaN,0.0,1.0


In [ ]:
pool[pool['ads_idx'] == 349591]

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,ctit_mean,ctit_median,ads_rejoin_type,contract_price,media_price,first_click,last_click,ads_name,ads_sdate,expire,days_active,daily_avg_conversions,cvr,margin,roi,total_net_return,ads_level_detailed,daily_clicks,daily_users,scale_level,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
367,376,380,119959,4,2175,2819,1431,2,게임,1,2757800,1151.501048,162.0,NONE,170,120,2025-07-28 01:27:04,2025-08-25 11:19:23,70억 좀비 - 라스트워,2024-11-30 01:00:00,0,29,49.344828,0.5,50,0.4,71550,LARGE,97.206897,75.0,MEGA,HIGH,MEGA_HIGH,3,3,3,3,0,12,LARGE,0,NaN,0.0,1.0


In [30]:
new

,ads_idx,ads_type,ads_category,ads_name,ads_os_type,ads_contract_price,ads_media_price,ads_reward_price,ads_rejoin_type,domain
0,500000,12,11,청정원 선물세트,7,6181,4000,4000,REJOINABLE,식음료
1,500001,3,4,옹짬뽕 포항양덕본점,7,18,14,12,ADS_CODE_DAILY_UPDATE,식당/카페
2,500002,3,4,추석선물세트 (리뷰 전체보기 클릭 후 URL 제출),7,17,8,8,NONE,식당/카페
3,500003,11,3,네이버 뉴스 제주뉴스 구독,7,230,160,180,NONE,미디어/컨텐츠
4,500004,7,3,폴라베어 인스타그램 구독,1,230,160,180,NONE,커머스
5,500005,3,7,스마트페이 계좌개설,7,6400,4800,4800,NONE,금융
6,500006,3,5,토리 키우기 (Lv.3까지),1,1600,1200,1200,NONE,게임
7,500007,3,5,드래곤사가 신규 영웅 뽑기,2,6700,5000,5000,NONE,게임


In [3]:
media_performance = pd.read_csv("/Users/sparta/Documents/GitHub/final_project/data/media_performance_classification.csv")

In [4]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None):
    """
    특정 광고의 매체별 성과를 분석하는 함수
    """
    
    # 1. 해당 광고의 데이터가 있는지 확인
    ads_data = click_data[click_data['ads_idx'] == ads_idx]
    if len(ads_data) == 0:
        print(f"광고 {ads_idx}에 대한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 2. 기본 성과 데이터 추출
    ads_performance = ads_data.groupby(['ads_idx', 'mda_idx']).agg({
        'click_key': 'count',
        'conversion': 'sum',
        'contract_price': 'first',
        'media_price': 'first',
        'domain': 'first',
        'ads_category': 'first'
    }).reset_index()
    
    # 컬럼명 변경
    ads_performance.columns = ['ads_idx', 'mda_idx', 'total_clicks', 'total_conversions', 
                              'contract_price', 'media_price', 'domain', 'ads_category']
    
    # 전환율 및 수익 계산
    ads_performance['cvr'] = (
        ads_performance['total_conversions'] / ads_performance['total_clicks']
    ).round(4)
    
    ads_performance['profit_per_conversion'] = (
        ads_performance['contract_price'] - ads_performance['media_price']
    )
    ads_performance['total_profit'] = (
        ads_performance['total_conversions'] * ads_performance['profit_per_conversion']
    )
    
    # 3. 날짜 범위 및 활동일 계산
    click_data_copy = click_data.copy()
    if not pd.api.types.is_datetime64_any_dtype(click_data_copy['click_date']):
        click_data_copy['click_date'] = pd.to_datetime(click_data_copy['click_date'])
    
    ads_activity = (
        click_data_copy.loc[click_data_copy['ads_idx'] == ads_idx]
                      .groupby('mda_idx')['click_date']
                      .agg(first_click='min', last_click='max')
                      .reset_index()
    )
    
    ads_activity['days_active_calc'] = (
        (ads_activity['last_click'] - ads_activity['first_click']).dt.days + 1
    )
    
    # 4. 데이터 병합
    merged = ads_performance.merge(
        ads_activity[['mda_idx', 'first_click', 'last_click', 'days_active_calc']],
        on='mda_idx', how='left'
    )
    
    # 5. 일평균 지표 계산
    merged['daily_clicks'] = merged['total_clicks'] / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit'] = merged['total_profit'] / merged['days_active_calc']
    
    # 6. 배분 그룹 분류 (데이터가 충분한 경우에만)
    if len(merged) > 1:  # 최소 2개 이상의 매체가 있어야 중앙값 계산이 의미있음
        profit_median = merged['daily_profit'].median()
        conv_median = merged['daily_conversions'].median()
        
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분',
            '잘못 배분'
        )
        # 결과 정렬
        result = merged.sort_values(['배분그룹', 'daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        result = merged.reset_index(drop=True)
    
    return result

In [5]:
ads_446909_analysis = analyze_ads_performance(28947, click)

In [6]:
ads_446909_analysis

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,28947,294,165,53,230,180,채용,3,0.3212,50,2650,2025-07-27 19:19:34,2025-08-25 08:53:19,29,5.689655,1.827586,91.379310,잘 배분
1,28947,246,1,1,230,180,채용,3,1.0000,50,50,2025-08-07 15:08:24,2025-08-07 15:08:24,1,1.000000,1.000000,50.000000,잘 배분
2,28947,270,58,12,230,126,채용,3,0.2069,104,1248,2025-07-28 00:33:28,2025-08-24 11:03:29,28,2.071429,0.428571,44.571429,잘 배분
3,28947,337,7,6,230,180,채용,3,0.8571,50,300,2025-08-06 10:04:03,2025-08-21 11:11:04,16,0.437500,0.375000,18.750000,잘 배분
4,28947,54,25,6,230,180,채용,3,0.2400,50,300,2025-07-29 20:03:22,2025-08-22 15:59:09,24,1.041667,0.250000,12.500000,잘못 배분
5,28947,761,37,7,230,180,채용,3,0.1892,50,350,2025-07-26 23:51:34,2025-08-24 20:47:05,29,1.275862,0.241379,12.068966,잘못 배분
6,28947,30,7,5,230,180,채용,3,0.7143,50,250,2025-07-27 20:58:53,2025-08-17 02:31:44,21,0.333333,0.238095,11.904762,잘못 배분
7,28947,1037,2,0,230,180,채용,3,0.0000,50,0,2025-08-21 17:16:53,2025-08-21 17:18:16,1,2.000000,0.000000,0.000000,잘못 배분


In [7]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_pool.csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (4714, 44)


,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
0,0,0,6508,1,1,2,0,10,금융,7,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
1,1,1,6985,2,30,46,0,7,금융,7,...,0,0,3,0,5,MEDIUM,0,NaN,0.0,0.0
2,2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,...,0,0,3,0,4,SMALL,0,NaN,0.0,1.0
3,3,3,9264,1,1,1,0,10,금융,7,...,0,0,0,0,1,SMALL,0,NaN,0.0,1.0
4,4,4,9716,2,4,6,1,8,생활,2,...,0,1,1,0,4,SMALL,0,NaN,0.0,1.0


In [8]:
pool[pool['ads_idx'] == 157236]

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
449,461,481,157236,7,119,167,72,4,식당/카페,7,...,1,1,3,0,8,LARGE,0,"562,563",1.0,1.0


In [9]:
pd.set_option('display.max_rows', None)        # 모든 행 표시
pd.set_option('display.max_columns', None)     # 모든 열 표시
pd.set_option('display.width', 0)              # 가로폭 자동(줄바꿈 안 함)
pd.set_option('display.max_colwidth', None)    # 긴 텍스트 자르지 않음
pd.set_option('display.precision', 12)  

In [10]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')
pd.reset_option('display.precision')
pd.options.display.float_format = None
np.set_printoptions(threshold=1000, linewidth=75)


In [11]:
pool[pool['ads_size'] == 'LARGE']

,Unnamed: 0.1,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,...,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster,mda_idx_arr,M,A
5,5,5,9935,10,524,889,24,8,금융,7,...,0,2,4,0,9,LARGE,0,NaN,0.0,1.0
26,29,29,13209,14,758,2162,169,8,금융,2,...,1,3,4,0,12,LARGE,0,22,1.0,1.0
27,30,30,13927,2,2069,2808,1032,1,생활,1,...,3,3,3,0,11,LARGE,0,NaN,0.0,1.0
31,34,34,14074,15,186,558,33,8,금융,2,...,1,2,4,0,11,LARGE,0,NaN,0.0,1.0
34,37,37,14405,9,317,505,40,10,미디어/컨텐츠,7,...,1,2,4,0,10,LARGE,0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701,4758,9504,446891,2,27,30,25,4,뷰티,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4704,4761,9507,446894,2,20,22,17,4,커머스,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4706,4763,9509,446896,2,39,48,39,4,커머스,7,...,2,2,0,2,8,LARGE,0,NaN,0.0,0.0
4709,4766,9512,446899,2,52,63,49,4,커머스,7,...,3,2,0,2,9,LARGE,0,NaN,0.0,0.0


In [12]:
def analyze_ads_performance(ads_idx, click_data, media_portfolio=None, mda_idx=None):
    """
    특정 광고/매체 조합의 매체별 성과를 분석
    - ads_idx: 광고 ID (None이면 무시)
    - mda_idx: 매체사 ID (None이면 무시)
    """
    df = click_data.copy()

    # 1) 필터링: 광고, 매체 중 지정된 것만 적용
    if ads_idx is not None and not (isinstance(ads_idx, float) and pd.isna(ads_idx)):
        df = df[df['ads_idx'] == int(ads_idx)]
    if mda_idx is not None and not (isinstance(mda_idx, float) and pd.isna(mda_idx)):
        df = df[df['mda_idx'] == int(mda_idx)]

    if len(df) == 0:
        target = f"ads_idx={ads_idx}" if ads_idx is not None else ""
        target += f" mda_idx={mda_idx}" if mda_idx is not None else ""
        print(f"해당 조건({target.strip()})에 대한 데이터가 없습니다.")
        return pd.DataFrame()

    # 2) 기본 성과 집계 (항상 ads_idx × mda_idx 단위)
    agg = df.groupby(['ads_idx', 'mda_idx']).agg(
        total_clicks   = ('click_key', 'count'),
        total_conversions = ('conversion', 'sum'),
        contract_price = ('contract_price', 'first'),
        media_price    = ('media_price', 'first'),
        domain         = ('domain', 'first'),
        ads_category   = ('ads_category', 'first'),
    ).reset_index()

    # 전환율/수익
    agg['cvr'] = (agg['total_conversions'] / agg['total_clicks']).replace([np.inf, -np.inf], np.nan).fillna(0).round(4)
    agg['profit_per_conversion'] = (agg['contract_price'] - agg['media_price'])
    agg['total_profit'] = agg['total_conversions'] * agg['profit_per_conversion']

    # 3) 활동 기간(first/last click) 및 일평균 지표
    if not pd.api.types.is_datetime64_any_dtype(df['click_date']):
        df['click_date'] = pd.to_datetime(df['click_date'], errors='coerce')

    act = (df.groupby(['ads_idx','mda_idx'])['click_date']
             .agg(first_click='min', last_click='max')
             .reset_index())
    act['days_active_calc'] = (act['last_click'] - act['first_click']).dt.days + 1
    act['days_active_calc'] = act['days_active_calc'].clip(lower=1)  # 0일 방지

    merged = agg.merge(act, on=['ads_idx','mda_idx'], how='left')

    merged['daily_clicks']      = merged['total_clicks']      / merged['days_active_calc']
    merged['daily_conversions'] = merged['total_conversions'] / merged['days_active_calc']
    merged['daily_profit']      = merged['total_profit']      / merged['days_active_calc']

    # 4) (옵션) media_portfolio 붙이기
    if media_portfolio is not None and 'mda_idx' in media_portfolio.columns:
        merged = merged.merge(media_portfolio, on='mda_idx', how='left')

    # 5) 배분그룹 (행이 2개 이상일 때만)
    if len(merged) > 1:
        profit_median = merged['daily_profit'].median()
        conv_median   = merged['daily_conversions'].median()
        merged['배분그룹'] = np.where(
            (merged['daily_profit'] >= profit_median) & (merged['daily_conversions'] >= conv_median),
            '잘 배분', '잘못 배분'
        )
        merged = merged.sort_values(['배분그룹','daily_profit'], ascending=[True, False]).reset_index(drop=True)
    else:
        merged['배분그룹'] = '분류불가'
        merged = merged.reset_index(drop=True)

    return merged

# 편의용: 매체사 ID만으로 분석 (여러 광고가 그 매체에 집행된 경우 광고별로 한 줄씩 나옵니다)
def analyze_media_performance(mda_idx, click_data, media_portfolio=None, ads_idx=None):
    """
    mda_idx 기준 분석 (ads_idx를 추가로 주면 특정 광고만 필터)
    """
    return analyze_ads_performance(
        ads_idx=ads_idx,
        click_data=click_data,
        media_portfolio=media_portfolio,
        mda_idx=mda_idx
    )


In [45]:
# 1) 기존처럼 광고 기준
ads_28947 = analyze_ads_performance(349591, click)

In [46]:
ads_28947

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,349591,651,28,2,13600,10200,게임,5,0.0714,3400,6800,2025-07-31 12:18:04,2025-08-21 01:59:03,21,1.333333,0.095238,323.809524,잘 배분
1,349591,270,40,1,13600,7140,게임,5,0.0250,6460,6460,2025-07-29 12:29:52,2025-08-25 09:28:33,27,1.481481,0.037037,239.259259,잘 배분
2,349591,337,7,1,13600,10200,게임,5,0.1429,3400,3400,2025-07-26 19:47:10,2025-08-12 00:37:09,17,0.411765,0.058824,200.000000,잘 배분
3,349591,246,1,0,13600,10200,게임,5,0.0000,3400,0,2025-07-28 19:16:45,2025-07-28 19:16:45,1,1.000000,0.000000,0.000000,잘 배분
4,349591,281,1,0,13600,10200,게임,5,0.0000,3400,0,2025-08-17 00:13:43,2025-08-17 00:13:43,1,1.000000,0.000000,0.000000,잘 배분
5,349591,563,24,0,13600,10200,게임,5,0.0000,3400,0,2025-07-27 00:03:06,2025-08-24 13:34:19,29,0.827586,0.000000,0.000000,잘 배분
6,349591,643,31,0,13600,10200,게임,5,0.0000,3400,0,2025-07-27 14:33:34,2025-08-21 12:23:54,25,1.240000,0.000000,0.000000,잘 배분
7,349591,790,2,0,13600,10200,게임,5,0.0000,3400,0,2025-08-05 18:11:35,2025-08-22 18:40:59,18,0.111111,0.000000,0.000000,잘 배분


In [42]:
ads_34045 = analyze_ads_performance(34045, click)

In [43]:
ads_34045

,ads_idx,mda_idx,total_clicks,total_conversions,contract_price,media_price,domain,ads_category,cvr,profit_per_conversion,total_profit,first_click,last_click,days_active_calc,daily_clicks,daily_conversions,daily_profit,배분그룹
0,34045,14,11382,457,230,160,미디어/컨텐츠,3,0.0402,70,31990,2025-07-26 06:16:06,2025-08-25 11:21:18,31,367.161290,14.741935,1031.935484,잘 배분
1,34045,761,4075,209,230,160,미디어/컨텐츠,3,0.0513,70,14630,2025-07-26 06:32:18,2025-08-25 11:17:02,31,131.451613,6.741935,471.935484,잘 배분
2,34045,30,200,23,230,160,미디어/컨텐츠,3,0.1150,70,1610,2025-07-29 11:19:58,2025-08-24 22:20:47,27,7.407407,0.851852,59.629630,잘 배분
3,34045,270,109,14,230,112,미디어/컨텐츠,3,0.1284,118,1652,2025-07-27 09:02:50,2025-08-25 11:17:03,30,3.633333,0.466667,55.066667,잘 배분
4,34045,583,288,17,230,160,미디어/컨텐츠,3,0.0590,70,1190,2025-07-26 07:20:55,2025-08-24 22:24:52,30,9.600000,0.566667,39.666667,잘 배분
5,34045,482,3,1,230,160,미디어/컨텐츠,3,0.3333,70,70,2025-08-15 20:44:35,2025-08-18 12:12:00,3,1.000000,0.333333,23.333333,잘못 배분
6,34045,246,8,2,230,160,미디어/컨텐츠,3,0.2500,70,140,2025-08-07 13:45:58,2025-08-24 21:27:59,18,0.444444,0.111111,7.777778,잘못 배분
7,34045,337,8,3,230,160,미디어/컨텐츠,3,0.3750,70,210,2025-07-26 09:53:58,2025-08-23 16:14:22,29,0.275862,0.103448,7.241379,잘못 배분
8,34045,790,9,1,230,160,미디어/컨텐츠,3,0.1111,70,70,2025-07-29 12:51:26,2025-08-19 21:35:14,22,0.409091,0.045455,3.181818,잘못 배분
9,34045,1037,1,0,230,160,미디어/컨텐츠,3,0.0000,70,0,2025-07-26 11:48:06,2025-07-26 11:48:06,1,1.000000,0.000000,0.000000,잘못 배분


In [15]:
# 3) 특정 광고 + 특정 매체만 보고 싶을 때(정확히 한 줄 나옴)
one_pair = analyze_ads_performance(28947, click, mda_idx=123)

해당 조건(ads_idx=28947 mda_idx=123)에 대한 데이터가 없습니다.


In [16]:
media_portfolio[media_portfolio['mda_idx'].isin([980])]

,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
146,146,980,1,17302,5993,2025-07-26 00:01:09,2025-08-25 09:44:32,31,193.322581,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.333333,268956541


In [22]:
media_performance[media_performance['mda_idx']==14]

,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,MEDIUM,MEGA,SMALL,total_ads,MEGA_ratio,LARGE_ratio,MEDIUM_ratio,SMALL_ratio,ads_category_0_pct,ads_category_1_pct,ads_category_2_pct,ads_category_3_pct,ads_category_4_pct,ads_category_5_pct,ads_category_6_pct,ads_category_7_pct,ads_category_8_pct,ads_category_10_pct,ads_category_11_pct,ads_category_13_pct,ads_os_type_1_pct,ads_os_type_2_pct,ads_os_type_3_pct,ads_os_type_7_pct,domain_게임_pct,domain_교육_pct,domain_금융_pct,domain_기타_pct,domain_미디어/컨텐츠_pct,domain_뷰티_pct,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
1,1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,5,22,1,59,37.3,52.5,8.5,1.7,0.0,20.338983,18.644068,8.474576,0.0,25.423729,0.0,3.389831,22.033898,1.694915,0.0,0.0,42.372881,38.983051,0.0,18.644068,44.067797,0.0,11.864407,3.389831,8.474576,5.084746,0.0,13.559322,0.0,3.389831,0.0,0.0,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형


In [18]:
media_performance[media_performance['mda_idx'].isin([496, 270, 22, 761, 343])]

,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,...,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
3,3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,...,1.204819,7.831325,4.819277,1.204819,7.228916,2541770,0.422674,2025-08-25 11:14:50,대량처리형,대량처리형
14,14,14,270,3146,61793,29548,2025-07-26 00:01:09,2025-08-25 11:23:53,31,953.161290,...,2.776235,0.943920,6.496391,0.333148,25.818989,-144697,0.478177,2025-08-25 11:23:53,대량처리형,대량처리형
25,25,25,343,75814,181809,90504,2025-07-26 00:34:21,2025-08-25 11:21:25,31,2919.483871,...,0.000000,0.000000,0.000000,0.000000,11.764706,3931080,0.497797,2025-08-25 11:21:25,안정공급형(후보),안정공급형(후보)
46,46,46,496,80048,127975,26418,2025-07-26 00:06:05,2025-08-25 11:24:01,31,852.193548,...,0.000000,1.851852,3.703704,1.851852,7.407407,1639389,0.206431,2025-08-25 11:24:01,안정공급형,안정공급형
105,105,105,761,9314,71853,13419,2025-07-26 00:13:29,2025-08-25 11:17:02,31,432.870968,...,0.980392,3.676471,3.921569,0.735294,2.941176,781705,0.186756,2025-08-25 11:17:02,안정공급형,안정공급형


In [19]:
pd.set_option("display.max_rows", None)    # 행 전체 출력
pd.set_option("display.max_columns", None) # 열 전체 출력
pd.set_option("display.width", None)       # 줄 바꿈 없이 가로로 다 보여줌
pd.set_option("display.max_colwidth", None) # 셀 내용 잘리지 않게